In [5]:
from faker import Faker
import json
import sqlite3
import random
import secrets

In [6]:
# fetch the products id, names, price from the database
def fetch_products():
    conn = sqlite3.connect('database/database.db')
    c = conn.cursor()
    c.execute('SELECT * FROM pricings')
    pricings_table = c.fetchall()
    conn.close()
    return pricings_table

# fetch the customers id, names from the database
def fetch_customers():
    conn = sqlite3.connect('database/database.db')
    c = conn.cursor()
    c.execute('SELECT * FROM customers')
    customers_table = c.fetchall()
    conn.close()
    return customers_table

# randomly select a customer id from the customers table
def random_customer_id(customers_table):
    return random.choice(customers_table)[0]

# from the customer id get the customer type
def get_customer_type(customers_table, customer_id):
    for customer in customers_table:
        if customer[0] == customer_id:
            return customer[4]

# randomly select a product id from the pricings table
def random_product_id(pricings_table):
    return random.choice(pricings_table)[1]

# get the product name from the pricing table
def get_product_name(pricings_table, product_id):
    for product in pricings_table:
        if product[1] == product_id:
            return product[2]

# get the product price from the pricing table based on the products id and type
def get_product_price(pricings_table, product_id, customer_type):
    for product in pricings_table:
        if product[1] == product_id and customer_type == "Business":
            return product[4]
        else : # personal
            return product[5]

# order id 36 hexes
def order_crypto_id():
    return secrets.token_hex(36)

# order quantity
def order_quantity():
    return random.randint(1, 15)

# order sku
def order_sku():
    sku = [
        'CTN', # cartons
        'OUT', # outer
        'BTL', # bottle
        'PKT', # packet
        'BDL' # bundle
    ]
    
    weight = [
        0.1,
        0.25,
        0.2,
        0.35,
        0.1
    ]
    
    return random.choices(sku, weight,k=1)[0]

# order status
def order_status():
    status = [
        'pending',
        'processing',
        'completed',
        'cancelled'
    ]
    
    weight = [
        0.3,
        0.2,
        0.4,
        0.1
    ]
    
    return random.choices(status, weight, k=1)[0]

# order date
def order_date():
    fake = Faker()
    date_obj = fake.date_this_year()
    return date_obj.strftime('%Y-%m-%d')

In [7]:
def create_order():
    products = fetch_products()
    product_id = random_product_id(products)
    amount = order_quantity()
    
    order = {
        'id': order_crypto_id(),
        'product_id': product_id,
        'name': get_product_name(products, product_id),
        'price': get_product_price(products, product_id),
        'total_price': get_product_price(products, product_id) * amount,
        'quantity': amount,
        'sku': order_sku(),
        'status': order_status(),
        'date': order_date()
    }
    return order

def generate_multiple_orders(n):
    orders = [create_order() for _ in range(n)]
    return json.dumps(orders, indent=4)

In [8]:
# insert the orders into the database
conn = sqlite3.connect('database/database.db')
c = conn.cursor()

c.execute(
    '''
    CREATE TABLE IF NOT EXISTS orders (
        id TEXT PRIMARY KEY,
        product_id TEXT,
        name TEXT,
        price REAL,
        total_price REAL,
        quantity INTEGER,
        sku TEXT,
        status TEXT,
        date DATE,
        FOREIGN KEY (product_id) REFERENCES products(id)
    )
    '''
)

json_data = generate_multiple_orders(300)

# save the json data as json
with open('json_data/orders.json', 'w') as f:
    f.write(json_data)
    
for order in json.loads(json_data):
    c.execute(
        '''
        INSERT INTO orders (id, product_id, name, price, total_price, quantity, sku, status, date)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''',
        (
            order['id'],
            order['product_id'],
            order['name'],
            order['price'],
            order['total_price'],
            order['quantity'],
            order['sku'],
            order['status'],
            order['date']
        )
    )
    
conn.commit()
conn.close()

CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    email TEXT NOT NULL,
    phone_number TEXT,
    address TEXT,
    city TEXT,
    state TEXT,
    postal_code TEXT,
    country TEXT,
    date_of_registration DATE
);

CREATE TABLE products (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_name TEXT NOT NULL,
    price DECIMAL(10, 2) NOT NULL,
    category_id INTEGER,
    FOREIGN KEY (category_id) REFERENCES categories(category_id)
);

CREATE TABLE categories (
    category_id INTEGER PRIMARY KEY AUTOINCREMENT,
    category_name TEXT NOT NULL
);

CREATE TABLE order_items (
    order_item_id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id INTEGER NOT NULL,
    product_id INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    price DECIMAL(10, 2) NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id)
);

CREATE TABLE payments (
    payment_id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id INTEGER NOT NULL,
    payment_method TEXT NOT NULL,
    payment_date DATE NOT NULL,
    amount DECIMAL(10, 2) NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id)
);

CREATE TABLE shipping (
    shipping_id INTEGER PRIMARY KEY AUTOINCREMENT,
    order_id INTEGER NOT NULL,
    shipping_date DATE,
    delivery_date DATE,
    shipping_cost DECIMAL(10, 2),
    shipping_address TEXT NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders(order_id)
);

CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY AUTOINCREMENT,
    customer_id INTEGER NOT NULL,
    order_date DATE NOT NULL,
    order_status TEXT NOT NULL,
    total_amount DECIMAL(10, 2) NOT NULL,
    payment_method TEXT NOT NULL,
    shipping_address TEXT NOT NULL,
    billing_address TEXT NOT NULL,
    shipping_cost DECIMAL(10, 2),
    discount DECIMAL(10, 2),
    tax DECIMAL(10, 2),
    notes TEXT,
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);